# **Práctica 3**: GANs
# Parte 1: GANs simples

Simplificación de:
https://medium.com/@mattiaspinelli/simple-generative-adversarial-network-gans-with-keras-1fe578e44a87


### Ejercicio prelab:
Lee y ejecuta el siguiente código

Importamos librerias

In [ ]:
import os
import numpy as np



from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.legacy import Adam

import matplotlib.pyplot as plt

#from IPython.core.debugger import Tracer

from PIL import Image


In [ ]:
plt.switch_backend('agg') # allows code to run without a system DISPLAY

Leemos datos MNIST

In [ ]:
# Datos
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
 
X_train = np.expand_dims(X_train, axis=3)


# Modelo

### Diseñamos modelo

Dimensiones y optimizador

In [ ]:
width=28
height=28
channels=1

in_shape = X_train.shape
in_shape = in_shape[1:]
OPTIMZADOR_ADAM = Adam(learning_rate=0.0002, beta_1=0.5, decay=8e-8)

Modelo del Generador

In [ ]:
model_gen = Sequential()
model_gen.add(Dense(256, input_shape=(100,)))
model_gen.add(LeakyReLU(alpha=0.2))
model_gen.add(BatchNormalization(momentum=0.8))
model_gen.add(Dense(512))
model_gen.add(LeakyReLU(alpha=0.2))
model_gen.add(BatchNormalization(momentum=0.8))
model_gen.add(Dense(1024))
model_gen.add(LeakyReLU(alpha=0.2))
model_gen.add(BatchNormalization(momentum=0.8))
model_gen.add(Dense(np.prod(in_shape), activation='tanh'))
model_gen.add(Reshape(in_shape))
model_gen.summary()
model_gen.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)

Modelo del discriminador

In [ ]:
model_Disc = Sequential()
model_Disc.add(Flatten(input_shape=in_shape))
model_Disc.add(Dense(128, input_shape=in_shape))
model_Disc.add(LeakyReLU(alpha=0.2))
model_Disc.add(Dense(64))
model_Disc.add(LeakyReLU(alpha=0.2))
model_Disc.add(Dense(1, activation='sigmoid'))
model_Disc.summary()
model_Disc.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, metrics=['accuracy'])

Modelo combinado

In [ ]:
model_gan = Sequential()
model_gan.add(model_gen)
model_gan.add(model_Disc)
model_gan.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)

model_gan.summary()

# Entrenamiento

In [ ]:
# Parámetros del entrenamiento
epochs = 5000
batch = 10

# Bucle entrenamiento

DD_loss = np.zeros((epochs,))
GG_loss = np.zeros((epochs,))

for cnt in range(epochs):

    ## Entrenamos discriminador
        # Imágenes reales
    random_index = np.random.randint(0, len(X_train) - np.int64(batch/2))
    legit_images = X_train[random_index : random_index + np.int64(batch/2)].reshape((np.int64(batch/2),)+in_shape)
        # Imágenes sintéticas
    gen_noise = np.random.normal(0, 1, (np.int64(batch/2), 100))
    syntetic_images = model_gen.predict(gen_noise)
        # Combinamos imágenes reales y sintéticas
    x_combined_batch = np.concatenate((legit_images, syntetic_images))
    y_combined_batch = np.concatenate((np.ones((np.int64(batch/2), 1)), np.zeros((np.int64(batch/2), 1))))
        # Entrenamos discriminador
    d_loss = model_Disc.train_on_batch(x_combined_batch, y_combined_batch)


    ## Entrenamos generador
        # Imágenes sintéticasnp.zeros
    noise = np.random.normal(0, 1, (batch, 100))
    y_mislabled = np.ones((batch, 1))
        # Entremaos generador
    g_loss = model_gan.train_on_batch(noise, y_mislabled)

    ## Evolución entrenamiento
    print ('epoch: %d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, d_loss[0], g_loss))

    DD_loss[cnt] = d_loss[0]
    GG_loss[cnt] = g_loss
    

Curvas de aprendizaje

In [ ]:
%matplotlib inline 
plt.figure
plt.plot(DD_loss)
plt.plot(GG_loss)

# Generar datos

In [ ]:
# Generamos imagen sintética
gen_noise = np.random.normal(0, 1, (np.int64(10), 100))
syntetic_images = model_gen.predict(gen_noise)

In [ ]:
# Mostramos imagen sintética
plt.imshow(syntetic_images[5,:,:,0],cmap='gray')
plt.axis('off')

In [ ]:
# Esto es una imagen real (por comparar)
plt.imshow(X_train[0,:,:,0],cmap='gray')
plt.axis('off')

# Ejercicios

### Ejercicio 1:

Modifica el código para incluir la generación de las etiquetas:

Modelo cGAN:

- En lugar de cargar solo las imágenes de MNIST, cargamos tanto las imágenes como las etiquetas.

- Al entrenar el generador, debemos proporcionar tanto ruido aleatorio como etiquetas generadas aleatoriamente

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf 
from keras.datasets import mnist
from tensorflow.keras import initializers
from keras.layers import LeakyReLU
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from tensorflow.keras.utils import plot_model
from keras.layers.convolutional import UpSampling2D, Conv2D

from keras.models import Sequential, Model
from keras.optimizers import Adam

In [ ]:
# Definir las variables relevantes
# Datos
# Load MNIST  
(X_train, y_train), (_, _) = mnist.load_data()
print('X_train shape 1 :', X_train.shape)

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)

print('X_train shape 2:', X_train.shape)

num_classes = 10
#y_train = tf.keras.utils.to_categorical(y_train, num_classes) # Convertir etiquetas en vectores one-hot.. Se puede hacer con embebing dentro de la red

# Definir dimensiones de entrada
in_shape = (28,28,1)
img_dim = np.prod (in_shape)
init = initializers.RandomNormal(stddev=0.01)
latent_dim = 100
num_classes = 10

#optimizador
OPTIMZADOR_ADAM = Adam(learning_rate=0.0002, beta_1=0.5, decay=8e-8)

# Visualizar imagenes-labels reales // imagenes-labels generaadas
def generate_images(generator, epoch):
    rows, cols = 10, 10
    noise  = np.random.normal (0, 1, (rows * cols, 100))
    labels = np.random.randint (0, num_classes, rows * cols).reshape(-1, 1)  #y_train [:rows * cols]
    images = generator.predict ([noise, labels])
     
    images = np.reshape(images, (images.shape[0], 28, 28, 1))
    images = 0.5 * images + 0.5 # escalar de [-1,1] a [0,1]
    images = np.uint8(255 * images) # escalar a [0, 255]
     
    labels_in = labels.ravel()
    
    for i in range(rows):
        row = images[i*cols:(i+1)*cols]
        
        row = np.concatenate(row, axis=1)
        
        plt.title(f'{labels_in[i*cols:(i+1)*cols]}')
        plt.imshow(row[:,:,0])
        plt.axis('off')
        #plt.title(f"Epoch {epoch} - Row {i}")
        plt.tick_params(axis='both',labelsize=14)
        plt.show()

In [ ]:
# ***********************************
# Definir modelo generador
# ***********************************
generator = Sequential()
# Input layer  
generator.add(Dense(128, input_shape=(latent_dim,), kernel_initializer=init))
generator.add(LeakyReLU(alpha = 0.2))
generator.add(BatchNormalization(momentum = 0.8))

generator.add(Dense(256, kernel_initializer = init))
generator.add(LeakyReLU(alpha = 0.2))
generator.add(BatchNormalization(momentum = 0.8))

generator.add(Dense(512, kernel_initializer = init))
generator.add(LeakyReLU(alpha = 0.2))
generator.add(BatchNormalization(momentum = 0.8))

# Output layer
generator.add(Dense(img_dim, activation = 'tanh'))

# se pasa también la eiqueta de la imagen
# Create label embeddings, one-hot (label)
label = Input(shape=(1,), dtype='int32')
label_embedding = Embedding(num_classes, latent_dim)(label)
label_embedding = Flatten()(label_embedding)

# latent space
z = Input(shape=(latent_dim,))

# Output image
img = generator(Multiply()([z, label_embedding]))

# Generator with condition input
generator = Model([z, label], img)
generator.summary()
plot_model(generator, to_file='cgan_generator.png', show_shapes=True)
# ***********************************
# Definir modelo discriminador
# ***********************************

# Discriminator network
discriminator = Sequential()
 
# Input layer  
 
discriminator.add(Flatten(input_shape = in_shape))
discriminator.add(Dense(128, kernel_initializer=init))
discriminator.add(LeakyReLU(alpha=0.2))
discriminator.add(Dense(64, kernel_initializer=init))
discriminator.add(LeakyReLU(alpha=0.2))
# Output layer
discriminator.add(Dense(1, activation='sigmoid'))

# se pasa también la eiqueta de la imagen
label_d = Input(shape=(1,), dtype='int32')
label_embedding_d = Embedding(num_classes, img_dim)(label_d)
label_embedding_d = Flatten()(label_embedding_d)
label_embedding_d = Reshape((28, 28, 1))(label_embedding_d)
 
# image dimension 28x28
# Image input
img_d = Input (shape = in_shape)
# Output image
validity = discriminator(Multiply()([img_d, label_embedding_d]))

# Discriminator with condition input
discriminator = Model([img_d, label_d], validity)
discriminator.summary()
plot_model(discriminator, to_file='cgan_discriminator.png', show_shapes=True)
# Compilar modelos
discriminator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, metrics=['accuracy'])
#generator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)

# ***********************************
# Definir modelo cGan
# ***********************************
gen_input = Input(shape=(latent_dim,))
gen_label = Input(shape=(1,), dtype='int32')
synthetic_image = generator([gen_input, gen_label])
synthetic_image_reshaped = Reshape((28, 28, 1))(synthetic_image)
discriminator.trainable = False
validity = discriminator([synthetic_image_reshaped, gen_label])
cGan = Model([gen_input, gen_label], validity)
cGan.summary()
plot_model(cGan, to_file='cgan_model.png', show_shapes=True)
cGan.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, metrics=['accuracy'])

In [ ]:
# Parámetros del entrenamiento
epochs = 2000 
batch_size = 64

# Bucle entrenamiento
DD_loss = np.zeros((epochs+1,))
GG_loss = np.zeros((epochs+1,))
 

for cnt in range(epochs + 1):
   
    # ***********************************
    # Paso 1: Entrenar el discriminador
    discriminator.trainable = True
    # ***********************************
    
    
    # Genera un conjunto de imágenes reales
    idx = np.random.randint(0, X_train.shape[0], size=batch_size)
    #idx0 = cnt*batch_size
    #idx1 = (cnt+1)*batch_size

    real_images = X_train[idx]#X_train[idx0:idx1]
    real_labels = y_train[idx].reshape(-1, 1)#y_train[idx0:idx1].reshape(-1, 1)

    #print(real_images.shape)
    #print(real_labels.shape)
    #print(real_labels)

    # Genera un conjunto de imágenes falsas
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
    fake_images = generator.predict([noise,fake_labels])
    #print(fake_images.shape)
    fake_images = np.reshape(fake_images, (fake_images.shape[0], 28, 28, 1))
    
    # Entrenamiento discriminador
    
    #print(real_images.shape)
    #print(real_labels.shape)

    #print(fake_images.shape)
    #print(fake_labels.shape)

    d_loss_real = discriminator.train_on_batch([real_images, real_labels], np.ones(batch_size))
    d_loss_fake = discriminator.train_on_batch([fake_images, fake_labels], np.zeros(batch_size))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ***********************************
    # Paso 2: Entrenar el generador
    discriminator.trainable = False
    # ***********************************
    gen_input  = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_labels = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
    y_gen = np.ones(batch_size)
    g_loss = cGan.train_on_batch([gen_input, gen_labels],y_gen)

    # Registrar el progreso del entrenamiento
    DD_loss[cnt] = d_loss[0]
    GG_loss[cnt] = g_loss [0]

    print(f'Epoch: {cnt}/{epochs} \t Discriminator Loss: {d_loss[0]} \t Generator Loss: {g_loss[0]}')
    # Imprimir progreso cada 100 iteraciones
    if cnt % 100 == 0:
        print(y_train.shape)
        generate_images(generator, cnt)

        samples = 10
        z = np.random.normal(loc=0, scale=1, size=(samples, latent_dim))
        labels = np.arange(0, 10).reshape(-1, 1)

        x_fake = generator.predict([z, labels])
        x_fake =  np.reshape(x_fake, (x_fake.shape[0], 28, 28, 1))
        for k in range(samples):
            plt.subplot(2, 5, k+1)
            plt.title(f'Label: {labels[k]}')
            plt.imshow(x_fake[k,:,:,0].reshape(28, 28), cmap='gray')
            plt.xticks([])
            plt.yticks([])

        plt.tight_layout()
        plt.show()

In [ ]:
# Generar imágenes con etiquetas aleatorias
n_to_show   = 10
latent_dim  = 100
num_classes = 10

labels = np.random.randint(0, num_classes, n_to_show).reshape(-1, 1) 
z = np.random.normal(loc=0, scale=1, size=(n_to_show, latent_dim))
labels = np.random.randint(0, num_classes, n_to_show).reshape(-1, 1) 

x_fake = generator.predict([z, labels])
x_fake =  np.reshape(x_fake, (x_fake.shape[0], 28, 28, 1))

# Mostrar imágenes generadas con sus etiquetas
fig, axs = plt.subplots(1, n_to_show, figsize=(12,4))
for i in range(n_to_show):
    axs[i].set_title(f'Label: {labels[i]}')
    axs[i].imshow(x_fake[i, :,:,0].reshape(28, 28), cmap='gray')
    axs[i].axis('off')
plt.show() 

In [ ]:
%matplotlib inline
plt.figure
plt.plot(DD_loss, label="Discriminator Loss")
plt.plot(GG_loss, label="Generator Loss")
plt.legend()

Vemos que con 1000 epocs ya sería suficiente para generar imagenes similares a las reales y no llega a detectar si son falsas o no.

# OTRO

In [ ]:
# Definir modelo generador
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Concatenate, Lambda

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
import numpy as np
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf 
from keras.datasets import mnist
from tensorflow.keras import initializers
from keras.layers import LeakyReLU
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from tensorflow.keras.utils import plot_model
from keras.layers.convolutional import UpSampling2D, Conv2D

from keras.models import Sequential, Model
from keras.optimizers import Adam
# Load MNIST  
(X_train, y_train), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3) # 28,28,1

# Definimos la dimensión de las etiquetas
num_classes = 10
 
# Definimos la dimensión de las imágenes
img_rows = 28
img_cols = 28
channels = 1
# Tamaño de entrada de las imágenes
in_shape = (img_rows, img_cols, channels)

img_dim = np.prod (in_shape)
 
# Tamaño de entrada de la etiqueta
label_shape = (1,)

# Definimos el número de neuronas en la capa oculta
hidden_dim = 128

# Función de activación para las capas densas
activation = 'relu'# 'LeakyRelu'

# Tamaño del vector de ruido
latent_dim = 100

def custom_loss(y_true, y_pred):
    # y_true es una lista con dos elementos: la imagen real y la etiqueta real
    # y_pred es una lista con dos elementos: la imagen generada y la etiqueta generada
    img_loss = tf.keras.losses.binary_crossentropy(y_true[0], y_pred[0])
    label_loss = tf.keras.losses.categorical_crossentropy(y_true[1], y_pred[1])

    return img_loss + label_loss

def cgan_loss(y_true, y_pred):
    # y_true es una lista con dos elementos: la imagen real y la etiqueta real
    # y_pred es una lista con dos elementos: la imagen generada y la etiqueta generada

    # Pérdida en la generación de imágenes
    img_loss = tf.keras.losses.binary_crossentropy(y_true[0], y_pred[0])

    # Pérdida en la generación de etiquetas
    label_loss = tf.keras.losses.binary_crossentropy(y_true[1], y_pred[1])

    # Pérdida total
    total_loss = img_loss + label_loss

    return total_loss



# Función para crear el discriminador
def build_discriminator(in_shape,img_dim, label_shape):
    # Input layer
    dis_input = Input(shape=in_shape)
    label_d = Input(shape=(1), dtype='int32')

    # Create label embeddings, one-hot (label)
    label_embedding_d = Embedding(num_classes, img_dim)(label_d)
    label_embedding_d = Flatten()(label_embedding_d)
    label_embedding_d = Reshape((28, 28, 1))(label_embedding_d)

    # Concatenate inputs
    concat_d = Multiply()([dis_input, label_embedding_d])

    # Hidden layers
    dis_layer1 = Dense(128, kernel_initializer='glorot_uniform')(Flatten()(concat_d))
    dis_layer1 = LeakyReLU(alpha=0.2)(dis_layer1)

    dis_layer2 = Dense(64, kernel_initializer='glorot_uniform')(dis_layer1)
    dis_layer2 = LeakyReLU(alpha=0.2)(dis_layer2)

    # Output layer
    dis_output = Dense(1, activation='sigmoid')(dis_layer2)

    # Modelo del discriminador
    # Discriminator with condition input
    discriminator = Model([dis_input, label_d], dis_output)

    # Compilar el modelo
    optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return discriminator
 
# Función para crear el generador
def build_generator(latent_dim, label_shape, in_shape):
    # Input layer
    gen_input = Input(shape=(latent_dim,))
    label = Input(shape=label_shape)

    # Create label embeddings, one-hot (label)
    label_embedding = Embedding(num_classes, latent_dim)(label)
    label_embedding = Flatten()(label_embedding)
    
    # Concatenar la etiqueta y el vector de ruido
    merged_layer = Multiply()([gen_input, label_embedding]) #Concatenate()([noise_input, label_input])

    gen_layer1 = Dense(128, kernel_initializer='glorot_uniform')(merged_layer)
    gen_layer1 = LeakyReLU(alpha=0.2)(gen_layer1)
    gen_layer1 = BatchNormalization(momentum=0.8)(gen_layer1)

    # Capas densas para la red neuronal del generador
    x = Dense(256, activation=activation)(gen_layer1)
    x = LeakyReLU(alpha = 0.2)(x)
    x = BatchNormalization(momentum = 0.8)(x)
                                 
    x = Dense(512, activation=activation)(x)
    x = LeakyReLU(alpha = 0.2)(x)
    x = BatchNormalization(momentum = 0.8)(x)
                                 
    # output                            
    img_label = Dense(np.prod(in_shape), activation='tanh')(x)

    # Modelo del generador
    generator_model = Model([gen_input, label], img_label)
    
    #plot_model(generator_model, to_file='cgan_generator_model.png', show_shapes=True)
    
    optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
    generator_model.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return generator_model

# Función para crear la cGAN
def build_cgan(generator_model, discriminator_model,latent_dim,label_shape):
    
    # Congelar el discriminador durante el entrenamiento del generador
    discriminator_model.trainable = False

    # Entrada para la imagen de ruido
    noise_input = Input(shape=(latent_dim,))
    print(noise_input.shape)
    # Entrada para la etiqueta
    label_input = Input(shape=label_shape)
    print(label_input.shape)
    # Generar imagen a partir de la imagen de ruido y la etiqueta
    img_output = generator_model([noise_input, label_input])
    img_output = Reshape((28, 28, 1))(img_output)
    print(img_output.shape)
    # El discriminador se entrena para distinguir entre imágenes reales y falsas
    validity = discriminator_model([img_output, label_input])

    # Modelo de la cGAN
    cgan_model = Model([noise_input, label_input], validity)

    # Compilar el modelo
    optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
    cgan_model.compile(loss='binary_crossentropy', optimizer=optimizer)

    return cgan_model



# Crear los modelos
discriminator = build_discriminator(in_shape, img_dim, 1)
plot_model(discriminator, to_file='cgan_discriminator_2.png', show_shapes=True)
discriminator.summary()

generator     = build_generator(latent_dim, label_shape, in_shape)
plot_model(generator, to_file='cgan_generator_2.png', show_shapes=True)
generator.summary()

cgan          = build_cgan(generator, discriminator,latent_dim,label_shape)
plot_model(cgan, to_file='cgan_cgan_2.png', show_shapes=True)
cgan.summary()

def plot_images(generator, noise_input,  epoch):
    # Generar imágenes a partir del ruido de entrada
 
    rows, cols = 10, 10
    noise  = np.random.normal(0, 1, (rows * cols, 100))
    #labels = tf.keras.utils.to_categorical(np.random.randint(0, num_classes, rows * cols).reshape(-1, 1))  #y_train [:rows * cols]
    labels =  np.random.randint(0, num_classes, rows * cols).reshape(-1, 1)
    images = generator.predict([noise, labels])
   
    images = np.reshape(images, (images.shape[0], 28, 28, 1))
    images = 0.5 * images + 0.5 # escalar de [-1,1] a [0,1]
    images = np.uint8(255 * images) # escalar a [0, 255]
    
    #label_output = Lambda(lambda x: np.argmax(x, axis=1))(g_labels)
    #label_output = tf.keras.utils.to_categorical(label_output, num_classes)
    label_output = labels.ravel()
    for i in range(rows):
        row = images[i*cols:(i+1)*cols]
        
        row = np.concatenate(row, axis=1)
        
        plt.title(f'{label_output[i*cols:(i+1)*cols]}')
        plt.imshow(row[:,:,0])
        plt.axis('off')
        #plt.title(f"Epoch {epoch} - Row {i}")
        plt.tick_params(axis='both',labelsize=14)
        plt.show()
        

def train(generator, discriminator, cgan, x_train, y_train, epochs, batch_size):
    dd_losses = []
    gg_losses = []
    for epoch in range(epochs):
        print("Epoch: ", epoch)

            # ---------------------
            #  Entrenamiento del Discriminador
            # ---------------------

            # Selecciona un conjunto aleatorio de imágenes reales
        
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        real_images = x_train[idx]
        real_labels = y_train[idx]
           
            # Genera un conjunto de imágenes falsas
            
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_labels = y_train[np.random.randint(0, y_train.shape[0], batch_size)] # np.random.randint(0, num_classes, batch_size)
        
        fake_images = generator.predict([noise, fake_labels])
        fake_images = np.reshape(fake_images, (fake_images.shape[0], 28, 28, 1))
        #label_output = Lambda(lambda x: np.argmax(x, axis=1))(gen_labels)
        #label_output = tf.keras.utils.to_categorical(label_output, num_classes)
        
        label_output = np.random.randint(0, num_classes, batch_size).reshape(-1, 1)
            # Entrenar la red adversaria (discriminator)
            
        discriminator.trainable = True
        discriminator_loss_real = discriminator.train_on_batch([real_images, real_labels], np.ones(batch_size))
        discriminator_loss_fake = discriminator.train_on_batch([fake_images, label_output], np.zeros(batch_size))
        d_loss = 0.5 * np.add(discriminator_loss_real, discriminator_loss_fake)
    
            # ---------------------
            #  Entrenamiento del Generador
            # ---------------------
        discriminator.trainable = False
            # Genera un conjunto de imágenes falsas
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_labels = y_train[np.random.randint(0,  y_train.shape[0], batch_size)]  #np.random.uniform(-1, 1, (batch_size, 1))
        
            # Etiquetas para la pérdida de la red generadora
        y_generator = np.ones(batch_size)
        g_loss = cgan.train_on_batch([noise,gen_labels], y_generator) #cgan.train_on_batch([noise, fake_labels], [np.ones(batch_size), fake_labels])
 
 
        print(f"Epoch: {epoch} \t Discriminator Loss: {d_loss} \t\t Generator Loss: {g_loss}")
        dd_losses.append(d_loss)
        gg_losses.append(g_loss)
        
        if epoch % 100 == 0:
            test_images = plot_images (generator,noise, epoch)
 
    return [dd_losses, gg_losses,generator,discriminator,cgan]



# Entrenamos el modelo
epochs = 5000
batch_size = 32

[dd_losses, gg_losses,generator,discriminator,cgan]  = train(generator, discriminator, cgan, X_train, y_train, epochs, batch_size)

In [ ]:
%matplotlib inline
plt.figure
plt.plot(DD_loss, label="Discriminator Loss")
plt.plot(GG_loss, label="Generator Loss")
plt.legend()

Vemos que con 500 epocs ya sería suficiente para generar imagenes similares a las reales y no llega a detectar si son falsas o no.

### Ejercicio 2:
Modifica el código para que use el dataset cifar10 en lugar del mnist 

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf 

from tensorflow.keras import initializers
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, Multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.layers import MaxPooling2D, concatenate
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import UpSampling2D, Conv2D

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.datasets import cifar10

In [ ]:
# Visualizar imagenes-labels reales 
def show_real_img (real_img,real_label,class_names,num_classes):
    # Vemos si se han cargado bien las imagenes
    fig = plt.figure(figsize=(8,3))
    for i in range(num_classes):
        ax = plt.subplot(2, 5, 1 + i, xticks=[], yticks=[])
        idx = np.where(real_label[:]==i)[0]
        features_idx = real_img [idx,::]
        img_num = np.random.randint(features_idx.shape[0])
        img = features_idx[img_num,::]
        ax.set_title(class_names[i])
        plt.imshow(img)

    plt.tight_layout()



# Visualizar imagenes-labels reales // imagenes-labels generaadas
def generate_images(generator, epoch):
    rows, cols = 10, 10
    noise  = np.random.normal(0, 1, (rows * cols, 100))
    labels = np.random.randint(0, num_classes, rows * cols).reshape(-1, 1)  #y_train [:rows * cols]
    images = generator.predict([noise, labels])
     
    images = np.reshape(images, (images.shape[0], 32, 32, 3))
    #images = 0.5 * images + 0.5 # escalar de [-1,1] a [0,1]
    images = np.uint8(255 * images) # escalar a [0, 255]
     
    labels_in = labels.ravel()
    
    for i in range(rows):
        row = images[i*cols:(i+1)*cols]
        
        row = np.concatenate(row, axis=1)
        print (labels_in[i*cols:(i+1)*cols])
        plt.title(f'{np.array(class_names)[labels_in[i*cols:(i+1)*cols]]}')
        plt.imshow(row[:,:,0])
        plt.axis('off')
        #plt.title(f"Epoch {epoch} - Row {i}")
        plt.tick_params(axis='both',labelsize=14)
        plt.show()

In [ ]:
#----------------------------------------
# Datos para el modelo y el entrenamiento
#----------------------------------------

# Definir dimensiones de entrada
in_shape =  (32, 32, 3)
img_dim = np.prod (in_shape)
init = initializers.RandomNormal(stddev=0.01)
latent_dim = 100
 
#optimizador
OPTIMZADOR_ADAM = Adam(learning_rate=0.0003, beta_1=0.5)



# Definir las variables relevantes
# Datos
# Load cifar10  
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')

num_classes = len(np.unique(y_train))
class_names = ['airplane','automobile','bird','cat','deer','dog',
                'frog','horse','ship','truck']

# Visualizar imagenes cargadas del dataset
show_real_img (X_train,y_train,class_names,num_classes)

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
 

# Normalizamos a 0-255
#X_train = X_train.astype('float32') / 255.0


print('X_train shape 2:', X_train.shape)
        


In [ ]:
 
# ***********************************
# Definir modelo generador
# ***********************************
generator = Sequential()

# Input layer  
generator.add(Dense(256, input_shape=(latent_dim,))) # , kernel_initializer=init
generator.add(BatchNormalization( momentum=0.8)) # momentum=0.8
generator.add(LeakyReLU(alpha=0.2))


generator.add(Dense(128 )) # , kernel_initializer=init
generator.add(BatchNormalization( momentum=0.8)) # momentum=0.8
generator.add(LeakyReLU(alpha=0.2))


generator.add(Dense(128 )) # , kernel_initializer=init
generator.add(BatchNormalization( momentum=0.8)) # momentum=0.8
generator.add(LeakyReLU(alpha=0.2))


generator.add(Dense(64))
generator.add(BatchNormalization()) # momentum=0.8
generator.add(LeakyReLU(alpha=0.2))

 
#generator.add(Dense(4096))
#generator.add(LeakyReLU(alpha=0.2))
#generator.add(BatchNormalization()) # momentum=0.8
 
#generator.add(Dense(8192))
#generator.add(LeakyReLU(alpha=0.2))
#generator.add(BatchNormalization()) # momentum=0.8

# Output layer
#generator.add(Flatten())
#generator.add(Dense(np.prod(in_shape), activation='sigmoid'))
generator.add(Dense(np.prod(in_shape), activation='tanh'))
generator.add(Reshape(in_shape))
generator.summary()
#generator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)

plot_model(generator, to_file='gan_generator_cifar10.png', show_shapes=True)

# ***********************************
# Definir modelo discriminador
# ***********************************

# Discriminator network
discriminator = Sequential()

# Input layer  
discriminator.add(Flatten(input_shape=in_shape))
discriminator.add(Dense(64, input_shape=in_shape)) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))
discriminator.add(Dense(128)) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))
discriminator.add(Dense(128)) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))
discriminator.add(Dense(256 )) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
discriminator.add(Dropout(0.3))
# Output layer
 
discriminator.add(Dense(1, activation='sigmoid'))
 
discriminator.summary()
# Compilar modelos
discriminator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, metrics=['accuracy'])

plot_model(discriminator, to_file='gan_discriminator_cifar10.png', show_shapes=True)
  
#generator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)
 
# ***********************************
# Definir modelo cGan
# ***********************************

  
z = Input(shape=(latent_dim,))
img = generator(z)

discriminator.trainable = False
validity = discriminator(img)

cGan = Model(z, validity)
cGan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))
cGan.summary()

In [ ]:
# Parámetros del entrenamiento

epochs= 10000
batch_size = 128
sample_interval= 100

# Bucle entrenamiento
valid = np.ones((batch_size, 1))
fake  = np.zeros((batch_size, 1))

DD_loss = np.zeros((epochs,))
GG_loss = np.zeros((epochs,))

for cnt in range(epochs):
    # ***********************************
    # Paso 1: Entrenar el discriminador
    discriminator.trainable = True
    # ***********************************
        
    # Genera un conjunto de imágenes reales
    idx = np.random.randint(0, X_train.shape[0], size=batch_size)

    real_images = X_train [idx]
    real_labels = y_train [idx]
    
    # Genera un conjunto de imágenes falsas
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    fake_images = generator.predict(noise)
    print ('fake_images:',fake_images.shape)
    fake_images = np.reshape(fake_images, (fake_images.shape[0], 32, 32, 3))
    
    print ('real_images:',real_images.shape)
    print ('valid:',valid.shape)
    # Entrenamiento discriminador
    d_loss_real = discriminator.train_on_batch(real_images, valid)
    d_loss_fake = discriminator.train_on_batch(fake_images, fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ***********************************
    # Paso 2: Entrenar el generador
    discriminator.trainable = False
    # ***********************************
    
    gen_input  = np.random.normal(0, 1, (batch_size, latent_dim))
    g_loss = cGan.train_on_batch(gen_input,valid) 

    # ***********************************
    # Paso 3: Evolución entrenamiento
    # **********************************
    DD_loss[cnt] = d_loss[0]
    GG_loss[cnt] = g_loss
    print ('epoch: %d/%d, [Discriminator :: d_loss: %f], [ Generator :: loss: %f]' % (cnt, epochs, d_loss[0], g_loss))
    # Mostrar el progreso del entrenamiento
    if cnt % sample_interval == 0:
              
        # Visualizar imagenes cargadas del dataset
        #r_img =  0.5 * real_images + 0.5
        #r_img = (r_img* 255).astype(np.uint8)
        
        #show_real_img (r_img, real_labels, class_names, num_classes)
        
        # Generar imágenes de muestra
        
         # Generate images from the generator
        r, c = 2, 5 # number of rows and columns of images to generate
        noise = np.random.normal(0, 1, (r * c, latent_dim))
        gen_imgs = generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        gen_imgs = (gen_imgs* 255).astype(np.uint8)
    
        # Plot generated images
        fig, axs = plt.subplots(r, c,figsize=(6,3.9),dpi=100)
        k = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[k, :,:,:])#.reshape(32, 32,3) )#.reshape(32, 32,3)
                axs[i,j].axis('off')
                k += 1
        plt.show()
#axs[i, j].imshow(gen_imgs[k,:,:,0].reshape(32, 32))

In [ ]:
%matplotlib inline
plt.figure
plt.plot(DD_loss, label="Discriminator Loss")
plt.plot(GG_loss, label="Generator Loss")
plt.legend()

un modelo malo. noo llega a converger. El generador no llega a interpretar y generar imagenes correctas y e disriminador en todo momento sabe si una imagen es real o falsa.

In [ ]:
# Generar imágenes con etiquetas aleatorias
n_to_show   = 10
latent_dim  = 100
num_classes = 10

z = np.random.normal(loc=0, scale=1, size=(n_to_show, latent_dim))
x_fake = generator.predict(z)
x_fake =  np.reshape(x_fake, (x_fake.shape[0], 32, 32, 3))
x_fake = 0.5 * x_fake + 0.5 # escalar de [-1,1] a [0,1]
x_fake = np.uint8(255 * x_fake).astype(np.uint8) # escalar a [0, 255]

# Mostrar imágenes generadas con sus etiquetas
fig, axs = plt.subplots(1, n_to_show, figsize=(20,6.9),dpi=100)
for i in range(n_to_show):
    axs[i].imshow(x_fake[i, :,:])
    axs[i].axis('off')
plt.show() 

## 2.1 .-  Cambiamos la red, compilamos la arquitectura del modelo y entrenamos de otra forma.

In [ ]:
 # ***********************************
# Definir modelo generador
# ***********************************
#optimizador
OPTIMZADOR_ADAM = Adam(learning_rate=0.003, beta_1=0.5,beta_2=0.99)

generator = Sequential()

# Input layer  
generator.add(Dense (256, input_shape=(latent_dim,))) # , kernel_initializer=init
generator.add(LeakyReLU (alpha=0.2))
generator.add(BatchNormalization (momentum=0.8)) # momentum=0.8
generator.add(Dropout(0.3))

generator.add(Dense (512 )) # , kernel_initializer=init
generator.add(LeakyReLU (alpha=0.2))
generator.add(BatchNormalization (momentum=0.8)) # momentum=0.8
generator.add(Dropout(0.3))
 
generator.add(Dense (1024 )) # , kernel_initializer=init
generator.add(LeakyReLU (alpha=0.2))
generator.add(BatchNormalization (momentum=0.8)) # momentum=0.8
generator.add(Dropout(0.3))

#generator.add(Dense(2048))
#generator.add(LeakyReLU(alpha=0.2))
#generator.add(BatchNormalization(momentum=0.8)) # momentum=0.8
#generator.add(Dropout(0.3))

#generator.add(Dense(4096))
#generator.add(LeakyReLU(alpha=0.2))
#generator.add(BatchNormalization()) # momentum=0.8
#generator.add(Dropout(0.3))

#generator.add(Dense(8192))
#generator.add(LeakyReLU(alpha=0.2))
#generator.add(BatchNormalization()) # momentum=0.8
#generator.add(Dropout(0.3))

# Output layer
#generator.add(Dense(np.prod(in_shape), activation='sigmoid'))
generator.add (Dense(np.prod (in_shape), activation='tanh'))
generator.add (Reshape (in_shape))
generator.summary ()

#generator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)
#plot_model(generator, to_file='gan_generator_cifar10.png', show_shapes=True)

# ***********************************
# Definir modelo discriminador
# ***********************************

# Discriminator network
discriminator = Sequential()

# Input layer  
discriminator.add(Flatten(input_shape=in_shape))

discriminator.add(Dense(512, input_shape=in_shape )) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))

discriminator.add(Dense(256)) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))

discriminator.add(Dense(128)) # , kernel_initializer=init
discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))

#discriminator.add(Dense(64 )) # , kernel_initializer=init
#discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8
#discriminator.add(Dropout(0.2))

#discriminator.add(Dense(32 )) # , kernel_initializer=init
#discriminator.add(LeakyReLU(alpha=0.2))
#discriminator.add(BatchNormalization()) # momentum=0.8

# Output layer
 
discriminator.add(Dense(1, activation='sigmoid'))
 
discriminator.summary()
# Compilar modelos
discriminator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, metrics=['accuracy'])

plot_model(discriminator, to_file='gan_discriminator_cifar10.png', show_shapes=True)
  
#generator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)
 
# ***********************************
# Definir modelo cGan
# ***********************************

  
z = Input(shape=(latent_dim,))
img = generator(z)

discriminator.trainable = False
validity = discriminator(img)

cGan = Model(z, validity)
cGan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.002, beta_1=0.5))
cGan.summary()

In [ ]:
epochs = 100
batch_size = 64
smooth = 0.1

real = np.ones(shape=(batch_size, 1))
fake = np.zeros(shape=(batch_size, 1))

d_loss = []
g_loss = []

for e in range(epochs + 1):
    for i in range(len(X_train) // batch_size):
        
        # Train Discriminator weights
        discriminator.trainable = True
        
        # Real samples
        X_batch = X_train[i*batch_size:(i+1)*batch_size]
        d_loss_real = discriminator.train_on_batch(x=X_batch,
                                                   y=real * (1 - smooth))
        
        # Fake Samples
        z = np.random.normal(loc=0, scale=1, size=(batch_size, latent_dim))
        X_fake = generator.predict_on_batch(z)
        d_loss_fake = discriminator.train_on_batch(x=X_fake, y=fake)
         
        # Discriminator loss
        d_loss_batch = 0.5 * (d_loss_real[0] + d_loss_fake[0])
        
        # Train Generator weights
        discriminator.trainable = False
        g_loss_batch = cGan.train_on_batch(x=z, y=real)

        print('epoch = %d/%d, batch = %d/%d, d_loss=%.3f, g_loss=%.3f' % (e + 1, epochs, i, len(X_train) // batch_size, d_loss_batch, g_loss_batch),100*' ')
    
    d_loss.append(d_loss_batch)
    g_loss.append(g_loss_batch)
    print('epoch = %d/%d, d_loss=%.3f, g_loss=%.3f' % (e + 1, epochs, d_loss[-1], g_loss[-1]), 100*' ')

    #if e % 10 == 0:
    samples = 10
    x_fake = generator.predict(np.random.normal(loc=0, scale=1, size=(samples, latent_dim)))

    for k in range(samples):
        plt.subplot(2, 5, k + 1, xticks=[], yticks=[])
        plt.imshow(((x_fake[k] + 1)* 127).astype(np.uint8))

    plt.tight_layout()
    plt.show()

In [ ]:
%matplotlib inline
plt.figure
plt.plot(d_loss, label="Discriminator Loss")
plt.plot(g_loss, label="Generator Loss")
plt.legend()

In [ ]:
# Generar imágenes con etiquetas aleatorias
n_to_show   = 10
latent_dim  = 100
num_classes = 10

z = np.random.normal(loc=0, scale=1, size=(n_to_show, latent_dim))
x_fake = generator.predict(z)
x_fake =  np.reshape(x_fake, (x_fake.shape[0], 32, 32, 3))
x_fake = 0.5 * x_fake + 0.5 # escalar de [-1,1] a [0,1]
x_fake = np.uint8(255 * x_fake).astype(np.uint8) # escalar a [0, 255]

# Mostrar imágenes generadas con sus etiquetas
fig, axs = plt.subplots(1, n_to_show, figsize=(20,6.9),dpi=100)
for i in range(n_to_show):
    axs[i].set_title(f'Label: {class_names[i]}')
    axs[i].imshow(x_fake[i, :,:])
    axs[i].axis('off')
plt.show() 

Tenemos peores resultados. Vemos que en este caso, con imagenes algo más complejas, utilizar exclusivamente capas lineales densas nos lleva a resultados no deseados. Probablemente habría que optimzar más los hiperparámetros y/o realizar mas emtrenamiento.

### Ejercicio 3:
Modifica el código para que use capas convolucionales en lugar de densas (en la medida de lo posible).

In [ ]:
import os
import numpy as np

import tensorflow as tf

from tensorflow.keras import initializers
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout,Conv2DTranspose,Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers.legacy import Adam
from keras.models import Sequential, Model
import matplotlib.pyplot as plt

#from IPython.core.debugger import Tracer

from PIL import Image

In [ ]:
# Visualizar imagenes reales 
def show_real_img (real_img):
    # Vemos si se han cargado bien las imagenes
    fig = plt.figure(figsize=(2,5))
    for i in range(10):
        ax = plt.subplot(2, 5, 1 + i, xticks=[], yticks=[])
        features_idx = real_img [i,::]
        plt.imshow(features_idx)

    plt.tight_layout()


# Datos
(X_train, _), (_, _) = mnist.load_data()

# Rescale -1 to 1
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = np.expand_dims(X_train, axis=3)


In [ ]:
# Definir la forma de entrada
width    = 28
height   = 28
channels = 1
in_shape = (width, height, channels)
# imagem dimension 28x28
img_dim    = np.prod( in_shape)
latent_dim = 100
init = initializers.RandomNormal (stddev=0.02)
# Definir el optimizador
OPTIMZADOR_ADAM = Adam (learning_rate = 0.002, beta_1 =0.5)#, beta_1=0.5b

In [ ]:
def build_generator_1(latent_dim, in_shape, img_dim, init, channels = 1,
                      kernel_size = (5, 5), strides = (1, 1), padding='same', 
                      use_bias = False):
    
    generator = Sequential ()

    generator.add (Dense(256*7*7, input_shape = (latent_dim,), 
                                     kernel_initializer = init))
    generator.add (BatchNormalization())
    generator.add (LeakyReLU())
    
    generator.add (Reshape((7,7,256)))
    

    generator.add(Conv2DTranspose(filters = 128, kernel_size = kernel_size, 
                                  strides = strides, padding = padding, use_bias = use_bias))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU())
    
    generator.add(Conv2DTranspose(filters = 64, kernel_size = kernel_size, 
                                  strides =(strides [0]*2, strides [1]*2), padding = padding, use_bias = use_bias))
    generator.add(BatchNormalization())
    generator.add(LeakyReLU())

    generator.add(Conv2DTranspose(filters=channels, kernel_size=kernel_size, 
                                  strides=(strides [0]*2, strides [1]*2), use_bias = use_bias,  padding=padding, activation='tanh'))



    return generator

#++++++++++++++++++++++++++++++++++++++++++++
#++++++++++++++++++++++++++++++++++++++++++++
#++++++++++++++++++++++++++++++++++++++++++++

def build_generator_2 (latent_dim,in_shape, img_dim, init, channels = 1,
                       kernel_size = 3, strides = (1, 1), padding='same', 
                       use_bias = False):
    
    generator = tf.keras.Sequential ()
    generator.add(Dense (7*7*256, use_bias = use_bias, input_shape = (latent_dim,), 
                                     kernel_initializer = init))
    generator.add(BatchNormalization (momentum = 0.8 ))
    generator.add(LeakyReLU (0.2))
    generator.add(Reshape ((7, 7, 256)))

    generator.add(Conv2DTranspose (128, kernel_size = kernel_size, 
                                      strides = strides, padding = padding, use_bias = use_bias))
    generator.add(BatchNormalization (momentum = 0.8 ))
    generator.add(LeakyReLU (0.2))

    generator.add(Conv2DTranspose (64, kernel_size = kernel_size, strides = (strides [0]*2, strides [1]*2), 
                                          padding = padding, use_bias = use_bias))
    generator.add(BatchNormalization (momentum = 0.8 ))
    generator.add(LeakyReLU (0.2))

    generator.add(Conv2DTranspose (channels, kernel_size = kernel_size, 
                                      strides = (strides [0]*2, strides [1]*2), padding = padding, use_bias = use_bias, activation = 'tanh'))
 
    generator_input  = Input(shape = (latent_dim,))
    generator_output = generator (generator_input)

    generator_model  = Model(generator_input, generator_output)
    return generator_model

# Definir el generador
#generator =  build_generator_1  (latent_dim, in_shape, img_dim, init, channels = 1, kernel_size = (5, 5), 
#                                strides = (1, 1), padding = 'same', use_bias = False)
generator =  build_generator_2 (latent_dim, in_shape, img_dim, init, channels = 1, kernel_size = 3, 
                                strides = (1, 1), padding = 'same', use_bias = False)
generator.summary()
generator.compile(loss='binary_crossentropy', optimizer = OPTIMZADOR_ADAM, metrics=['accuracy'])

In [ ]:

def build_discriminator_1(latent_dim,in_shape, img_dim, channels = 1, 
                          kernel_size = (5, 5), strides = (2,2), padding = 'same'):
    discriminator  = Sequential ()

    discriminator.add (Conv2D (filters = 64, kernel_size = kernel_size, strides = strides,
                               padding = padding, input_shape=in_shape))
    #discriminator.add(BatchNormalization()) # momentum=0.8                   
    discriminator.add (LeakyReLU ())
    discriminator.add(layers.Dropout (0.3))                   

    discriminator.add (Conv2D (filters=128, kernel_size = kernel_size, strides = strides, 
                             padding = padding))
    #discriminator.add(BatchNormalization()) # momentum=0.8
    discriminator.add(LeakyReLU())
    discriminator.add(Dropout(0.3))

    discriminator.add(Flatten())
    discriminator.add(Dense(1, activation='sigmoid'))

    return discriminator

def build_discriminator_2 (latent_dim,in_shape, img_dim,  init, channels = 1,
                          kernel_size=3, strides = (1, 1), padding= 'same'):
                       
    discriminator  = Sequential ()

    discriminator.add (Conv2D(filters=64, kernel_size=kernel_size, strides = strides, padding = padding,
                          input_shape = in_shape, kernel_initializer = init))
    discriminator.add (BatchNormalization (momentum=0.8 ))#
    discriminator.add (LeakyReLU (alpha = 0.2))
    discriminator.add (Dropout (0.3))
    
    discriminator.add (Conv2D (filters=128, kernel_size = kernel_size, strides = strides, padding = padding,
                              kernel_initializer = init))
    discriminator.add (BatchNormalization (momentum=0.8))#momentum=0.8
    discriminator.add (LeakyReLU(alpha = 0.2))
    discriminator.add (Dropout (0.3))
    
    discriminator.add (Conv2D(filters=256, kernel_size = kernel_size, strides = strides, padding = padding,
                            kernel_initializer = init))
    discriminator.add (BatchNormalization (momentum=0.8 ))#momentum=0.8
    discriminator.add (LeakyReLU (alpha = 0.2))
    discriminator.add (Dropout (0.3))

    discriminator.add (Flatten ())
    discriminator.add (Dense (channels, activation='sigmoid'))

    return discriminator

################################################
# Definir el discriminador
################################################´

#discriminator = build_discriminator_1 (latent_dim,in_shape, img_dim, channels= 1,   kernel_size = (5, 5), strides = (1, 1),   padding= 'same')
discriminator = build_discriminator_2 (latent_dim,in_shape, img_dim,  init, channels = 1, kernel_size = 3, strides = (2,2), padding='same')
discriminator.summary()

# Compilar el discriminador
discriminator.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM, metrics=['accuracy'])


In [ ]:
# Combinar el generador y el discriminador
model_gan = Sequential()
model_gan.add(generator)
model_gan.add(discriminator)

# Compilar el GAN
model_gan.compile(loss='binary_crossentropy', optimizer=OPTIMZADOR_ADAM)
model_gan.summary()

In [ ]:
# Definir el número de épocas y el tamaño del lote
num_epochs = 3000
batch_size = 128
sample_interval = 100
num_classes = 10
# Definir los vectores de seguimiento para la pérdida del generador y del discriminador
losses_g = []
losses_d = []
c_ones  = np.ones((batch_size, 1))
c_zeros = np.zeros((batch_size, 1))
# Entrenar el modelo
for epoch in range(num_epochs):
    discriminator.train = True
    # Seleccionar un conjunto aleatorio de imágenes de entrenamiento reales
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    real_images = X_train[idx]
    #show_real_img (real_images)
    # Generar un conjunto aleatorio de imágenes de entrenamiento falsas
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    noise = (noise - 0.5) * 2 # escalar los valores al rango de -1 a 1
    fake_images = generator.predict(noise)

    # Entrenar el discriminador en las imágenes reales y falsas
    d_loss_real = discriminator.train_on_batch(real_images, c_ones)
    d_loss_fake = discriminator.train_on_batch(fake_images, c_zeros)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    discriminator.train = False
    # Entrenar el generador para engañar al discriminador
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    noise = (noise - 0.5) * 2
    g_loss = model_gan.train_on_batch(noise, c_ones)
    
    # Guardar el seguimiento de las pérdidas del generador y del discriminador
    losses_g.append (g_loss)
    losses_d.append (d_loss[0])
    
    # Imprimir el progreso del entrenamiento
    print("Epoca: %d [Discriminador Loss: %f] [Generador Loss: %f]" % (epoch, d_loss[0], g_loss ))
    if epoch % sample_interval == 0:

        # Generar imágenes falsas a partir de vectores de ruido aleatorios
        noise = np.random.normal(0, 1, (num_classes, latent_dim))
        noise = (noise - 0.5) * 2
        images = generator.predict(noise)
 
             
        images = 0.5 * images + 0.5 # escalar de [-1,1] a [0,1]
        #images = np.uint8(255 * images) # escalar a [0, 255]
        # Mostrar las imágenes generadas
        r = 2
        c = 5
        fig, axs = plt.subplots(r, c,figsize=(11,6.9),dpi=100)
        count = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(images[count, :, :].reshape(28,28), cmap="gray") #, cmap='gray'
                axs[i,j].axis('off')
                count += 1
        plt.show()

In [ ]:
%matplotlib inline 
plt.figure
plt.plot (losses_g, label="Generator Loss")
plt.plot (losses_d, label="Discriminator Loss")
plt.legend()

Vemos que los números son más nitidos que en la red donde se utilizan sólo capas densas. y con esta configuracón de hiperparámetros y arquitectura d capsa se necesitan pocas epochs para conseguir un resultado aceptable. Asemás vemos que con unos 900 y 100 epochs el modelo converge, teniendo unos errores aprox. constantes.

# otro: Veamos con imagenes más complejas (Cifar-10).

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

from keras.datasets import cifar10
from keras.models import Sequential, Model
from keras.layers import Input, Dense, LeakyReLU, BatchNormalization, ReLU
from keras.layers import Conv2D, Conv2DTranspose, Reshape, Flatten, Dropout
from keras.optimizers import Adam
from keras import initializers
from keras.utils import plot_model, np_utils
from keras import backend as K

In [ ]:
 # load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

fig = plt.figure(figsize=(8,3))
for i in range(0, 10):
    plt.subplot(2, 5, 1 + i, xticks=[], yticks=[])
    plt.imshow(X_train[i])
    
plt.tight_layout()


In [ ]:
num_classes = len(np.unique(y_train))
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

fig = plt.figure(figsize=(8,3))
for i in range(num_classes):
    ax = plt.subplot(2, 5, 1 + i, xticks=[], yticks=[])
    idx = np.where(y_train[:]==i)[0]
    features_idx = X_train[idx,::]
    img_num = np.random.randint(features_idx.shape[0])
    img = features_idx[img_num,::]
    ax.set_title(class_names[i])
    plt.imshow(img)
    
plt.tight_layout()

In [ ]:
if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape (X_train.shape [0], 3, 32, 32)
    X_test  = X_test.reshape (X_test.shape [0], 3, 32, 32)
    input_shape = (3, 32, 32)
else:
    X_train = X_train.reshape (X_train.shape [0], 32, 32, 3)
    X_test  = X_test.reshape (X_test.shape [0], 32, 32, 3)
    input_shape = (32, 32, 3)
    
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical (y_train, num_classes)
Y_test  = np_utils.to_categorical (y_test, num_classes)

# the generator is using tanh activation, for which we need to preprocess 
# the image data into the range between -1 and 1.

X_train = np.float32(X_train)
X_train = (X_train / 255 - 0.5) * 2
X_train = np.clip(X_train, -1, 1)

X_test = np.float32(X_test)
X_test = (X_train / 255 - 0.5) * 2
X_test = np.clip(X_test, -1, 1)

print('X_train reshape:', X_train.shape)
print('X_test reshape:', X_test.shape)

In [ ]:
# latent space dimension
latent_dim = 100

init = initializers.RandomNormal (stddev = 0.02)

# Generator network
generator = Sequential()

# FC: 2x2x512
generator.add (Dense (4*4*512, input_shape = (latent_dim,)))# , kernel_initializer=init
generator.add (LeakyReLU (alpha = 0.2)) ## ReLU()
generator.add (Reshape ((4, 4, 512)))
#generator.add (BatchNormalization ())


# # Conv 1: 4x4x256
generator.add (Conv2DTranspose (128, kernel_size = 4, strides = 2, padding='same'))
#generator.add(BatchNormalization())
generator.add (LeakyReLU (alpha = 0.2))## ReLU() 

# Conv 2: 8x8x128
generator.add (Conv2DTranspose (128, kernel_size = 4, strides = 2, padding='same'))
#generator.add(BatchNormalization())
generator.add (LeakyReLU (alpha = 0.2))## ReLU() alpha = 0.2

# Conv 3: 16x16x64
#generator.add (Conv2DTranspose (128, kernel_size = 5, strides = 2, padding = 'same'))
#generator.add(BatchNormalization())
#generator.add (ReLU ( ))##LeakyReLU(0.2)

# Conv 4: 32x32x3
generator.add  (Conv2DTranspose (3, kernel_size = 3, strides = 2, padding = 'same',
                              activation = 'tanh'))
generator.summary()

In [ ]:
# imagem shape 32x32x3
img_shape = X_train[0].shape

# Discriminator network
discriminator = Sequential ()
# Conv 0:  32x32x3
discriminator.add (Conv2D (64, kernel_size = 3, strides = 2, padding = 'same',
                                                  input_shape = (img_shape))) #
discriminator.add (LeakyReLU (alpha = 0.2))

# Conv 1: 16x16x64
discriminator.add (Conv2D (128, kernel_size = 3, strides = 2, padding = 'same')) #input_shape=(img_shape), kernel_initializer=init
#discriminator.add BatchNormalization ())
discriminator.add (LeakyReLU (alpha = 0.2))#0.2
# discriminator.add(Dropout(0.4))

# Conv 2: 8x8x128
discriminator.add (Conv2D (128, kernel_size = 3, strides = 2, padding = 'same'))
#discriminator.add BatchNormalization ())
discriminator.add (LeakyReLU (alpha = 0.2))#0.2
 

# Conv 3: 4x4x256
discriminator.add (Conv2D (256, kernel_size = 3, strides = 2, padding = 'same'))
#discriminator.add(BatchNormalization())
discriminator.add (LeakyReLU (alpha = 0.2 ))#0.2

 

# FC: 2048
discriminator.add (Flatten ())
discriminator.add (Dropout (0.4))
# Output: 1
discriminator.add (Dense (1, activation='sigmoid'))

# Optimizer

discriminator.compile (Adam (learning_rate = 0.0002, beta_1 = 0.5), loss = 'binary_crossentropy',
                      metrics = ['accuracy'])

discriminator.summary()

In [ ]:
# d_g = discriminador(generador(z))
discriminator.trainable = False

z   = Input (shape = (latent_dim,))
img = generator (z)
decision = discriminator (img)
d_g = Model (inputs = z, outputs = decision)

d_g.compile (Adam (learning_rate = 0.0004, beta_1 = 0.5), loss = 'binary_crossentropy',
            metrics = ['accuracy'])

In [ ]:
epochs = 100
batch_size = 32
smooth = 0.1

real = np.ones (shape = (batch_size, 1))
fake = np.zeros (shape = (batch_size, 1))

d_loss = []
g_loss = []

for e in range (epochs + 1):
    for i in range (len (X_train) // batch_size):
        
        # Train Discriminator weights
        discriminator.trainable = True
        
        # Real samples
        X_batch = X_train [i*batch_size:(i+1)*batch_size]
        d_loss_real = discriminator.train_on_batch (x = X_batch, y = real * (1 - smooth))
        
        # Fake Samples
        z = np.random.normal (loc = 0, scale = 1, size = (batch_size, latent_dim))
        X_fake = generator.predict_on_batch (z)
        d_loss_fake = discriminator.train_on_batch(x=X_fake, y=fake)
         
        # Discriminator loss
        d_loss_batch = 0.5 * (d_loss_real[0] + d_loss_fake[0])
        
        # Train Generator weights
        discriminator.trainable = False
        g_loss_batch = d_g.train_on_batch (x=z, y=real)

        print(
            'epoch = %d/%d, batch = %d/%d, d_loss=%.3f, g_loss=%.3f' % (e + 1, epochs, i, len(X_train) // batch_size, d_loss_batch, g_loss_batch[0]),
            100*' ',
            end='\r'
        )
    
    d_loss.append (d_loss_batch)
    g_loss.append (g_loss_batch [0])
    print('epoch = %d/%d, d_loss=%.3f, g_loss=%.3f' % (e + 1, epochs, d_loss[-1], g_loss[-1]), 100*' ')

    if e % 2 == 0:
        samples = 10
        x_fake = generator.predict (np.random.normal (loc = 0, scale = 1, size = (samples, latent_dim)))

        for k in range (samples):
            plt.subplot (2, 5, k + 1, xticks=[], yticks=[])
            plt.imshow (((x_fake[k] + 1)* 127).astype (np.uint8))

        plt.tight_layout ()
        plt.show ()

In [ ]:
%matplotlib inline 
%matplotlib inline 
plt.figure ()
plt.plot (g_loss, label="Generator Loss")
plt.plot (d_loss, label="Discriminator Loss")
plt.legend()

In [ ]:
 # Generar imágenes con etiquetas aleatorias
n_to_show   = 10
latent_dim  = 100
num_classes = 10

z = np.random.normal(loc=0, scale=1, size=(n_to_show, latent_dim))
x_fake = generator.predict(z)
x_fake =  np.reshape(x_fake, (x_fake.shape[0], 32, 32, 3))
x_fake = 0.5 * x_fake + 0.5 # escalar de [-1,1] a [0,1]
x_fake = np.uint8(255 * x_fake).astype(np.uint8) # escalar a [0, 255]

# Mostrar imágenes generadas con sus etiquetas
fig, axs = plt.subplots(1, n_to_show, figsize=(20,6.9),dpi=100)
for i in range(n_to_show):
    axs[i].imshow(x_fake[i, :,:])
    axs[i].axis('off')
plt.show()